In [ ]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected

from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist


from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [ ]:
n_qubits = 5
n_meas_shots = 16000
n_build_shots = 16000
n_shots_qiskit_full = n_build_shots // (2 ** n_qubits) 
n_shots_qiskit_partial = n_build_shots // (n_qubits)



n_circuit_method_shots = n_meas_shots + n_build_shots
backend = provider.get_backend('ibmq_quito')

circuit = GHZ_prep(backend)

n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))

err_cmap = [[3, 4], [0, 4], [1, 4], [2, 4]]

## Qiskit Full Build

In [ ]:
full_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_full)

# n Circuits to execute

linear_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_partial)



## Run with repetitions
If everything above is working, let's crank it out a few times

In [ ]:
n_reps = 10
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[]
}

for _ in range(n_reps):

    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
    tpf_err.build(verbose=True)
    
    tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
    tpf.build(verbose=True)
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        GHZ_state_dist(bare_result)
    )
    results['full'].append(
        GHZ_state_dist(full_filter.apply(bare_result))
    )
    results['linear'].append(
            GHZ_state_dist(linear_filter.apply(bare_result))
    )
    results['cmc'].append(
            GHZ_state_dist(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
            GHZ_state_dist(tpf_err.apply(bare_result))
    )
    results['aim'].append(GHZ_state_dist(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(GHZ_state_dist(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(GHZ_state_dist(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))



In [ ]:
results

In [ ]:
ghz_quito_results = {'bare': [0.25956250000000003,
  0.265875,
  0.262625,
  0.26937500000000003,
  0.2646875,
  0.26387499999999997,
  0.2573125,
  0.249875,
  0.27362499999999995,
  0.2636875],
 'full': [0.029775313464926623,
  0.06647910744451674,
  0.04667775226332799,
  0.0332809168928796,
  0.05201336090306968,
  0.02816636396648242,
  0.042579175033529204,
  0.044728550884244545,
  0.04091080425915061,
  0.041340332580012795],
 'linear': [0.06685398871637849,
  0.10307654542937617,
  0.08341716484628997,
  0.0682755370919717,
  0.08884475703082517,
  0.061451295852230636,
  0.07950304239046785,
  0.08032007572549499,
  0.07567339834293518,
  0.07817281690779665],
 'aim': [0.27145833333333336,
  0.268125,
  0.26879166666666665,
  0.26529166666666665,
  0.26316666666666666,
  0.2535833333333333,
  0.25654166666666667,
  0.2699166666666667,
  0.26087499999999997,
  0.2625],
 'sim': [0.27584375,
  0.2750625,
  0.27134375,
  0.26821875,
  0.26899999999999996,
  0.26999999999999996,
  0.2675625,
  0.2629375,
  0.27165625,
  0.28025],
 'jigsaw': [0.20800916343216552,
  0.21512919397038882,
  0.25229220293853327,
  0.23102214725988712,
  0.24311394040792333,
  0.2042409427714018,
  0.166963203629511,
  0.16949084162224526,
  0.1937579000713494,
  0.22623957846867676],
 'cmc': [0.15388526001646624,
  0.16621548127143265,
  0.1570145400187437,
  0.16281618433360506,
  0.15862432406425914,
  0.1545232415778796,
  0.15268780422335748,
  0.14605907920957362,
  0.1701612808123521,
  0.15924838384240414], 
 'cmc_err':[0.2744379333219121,
  0.1695567970415347,
  0.16489292681487588,
  0.14460394095542106,
  0.15745310780670307,
  0.15711038837488828,
  0.1574260355967118,
  0.1669498632895472,
  0.17499063777335538]}

In [ ]:
res = ghz_quito_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')